## store LIWC word lists into pickle files

In [51]:
## store LIWC word lists into pickle files
## Each pickle is a list of strings

import os
import pickle
directory = '/Users/muriel820/Downloads/ml-project/liwc'
os.chdir(directory)
fnames = os.listdir(directory)
for f in fnames:
    if f.endswith('.txt'):
        complete_file_directory = directory+'/'+f
        p = open(complete_file_directory,'r').read()
        # delete the situation like :) and :(  
        p = re.sub(r'[\:\(\)]','',p)
        p = p.split('\n')
        name = f[:-4] + '.p'        
        pickle.dump(p, open(directory+'/'+name, "wb"))


In [2]:
import pickle
filler = pickle.load(open('/Users/muriel820/Downloads/ml-project/liwc/125-filler.p','rb'))
#filler

['anyway*',
 'blah',
 'dunno',
 'idk',
 'idontknow',
 'imean',
 'ohwell',
 'rr*',
 'whoa',
 'woah',
 "y'kn*",
 'yakno*',
 'ykn*',
 'youknow',
 'which were']

## Basic idea (1 paragrap - 1 category)

In [76]:
d = dict()
s = '("Feeder Funds"), anyway which were idk. idk organized as limited partnerships under Delaware law. Through various explanatory material, including offering memoranda, the Feeder Funds advised investors who purchased interests in the funds that the investors yielded exclusive control over investment decisions to the funds. The Feeder Funds invested the pooled capital obtained from their investors with BLMIS through securities accounts maintained only in the funds\' names. The Feeder Funds made deposits to and withdrawals from their BLMIS accounts, and they received account documentation, including account statements and trade confirmations, from BLMIS. As investors in the Spectrum Funds that invested in the Feeder Funds, appellants had no direct financial dealings with BLMIS, and no account information in BLMIS records identified them as BLMIS investors.'

sad_vocab = filler
for i in range(len(sad_vocab)):
    sad_vocab[i] = re.sub(r"\*","?=([,.?!\s]|[a-z]+)", sad_vocab[i])
#sad_vocab
r = re.compile("|".join(r"\b%s\b" % w for w in sad_vocab))
wordcount = Counter(re.findall(r, s))
#wordcount
sad = sum(wordcount.values())
d['sad']=sad
d

{'sad': 3}

## load LIWC pickles into memory as regex form

In [77]:
## load all liwc pickles into vocab_all as a list of regex pattern

from collections import Counter
import re

def regex_liwc_words(vocab):
    #
    # input: vocab(a list of strings)
    # return: regex expression of all words in the vocabulary
    #
    for i in range(len(vocab)):
        vocab[i] = re.sub(r"\*","([a-z]+|\s)", vocab[i])
    r = re.compile("|".join(r"\b%s\b" % w for w in vocab))
    return r


directory = '/Users/muriel820/Downloads/ml-project/liwc'
os.chdir(directory)
fnames = os.listdir(directory)
vocab_id = []
vocab_all = []
for f in fnames:
    if f.endswith('.p'):
        vocab_id.append(f[:-4])
        complete_file_directory = directory+'/'+f
        vocab = pickle.load(open(complete_file_directory,'rb'))
        vocab = regex_liwc_words(vocab)
        vocab_all.append(vocab)
# vocab_all[0]

re.compile(r"\ba however so would've\b|\babout howre somebod([a-z]+|\s) wouldn't\b|\babove hows somehow wouldnt\b|\babsolutely i someone([a-z]+|\s) wouldve\b|\babt i'd something([a-z]+|\s) y'all\b|\bacross i'd've somewhat y'all's\b|\bactually i'll somewhere ya\b|\bafter i'm soon ya'll([a-z]+|\s)\b|\bagain i've sooo([a-z]+|\s) yall\b|\bagainst id specifically yalls\b|\bahead idc still ye\b|\bain't idgaf stuff yet\b|\baint idk subsequently yinz([a-z]+|\s)\b|\balmost idontknow such you\b|\balong idve suddenly you'd\b|\balready if supposedly you'll\b|\balso ikr than you're\b|\baltho ily([a-z]+|\s) that you've\b|\balthough im that'd youd\b|\bam ima that'll youll\b|\bamid imean that's your\b|\bamidst imma thatd youre\b|\bamong([a-z]+|\s) immediately thatll yours\b|\ban in thats yourself\b|\band including the yourselves\b|\banother indeed thee youve\b|\banybod([a-z]+|\s) inside their([a-z]+|\s)\b|\banymore insides them\b|\banyone([a-z]+|\s) instead themself\b|\banything into themselves\b|\ban

## run through all paragraphs and store in zipfile

In [75]:
import os
from glob import glob
from zipfile import ZipFile
from collections import Counter
import pickle
import re
import shutil

def liwc_words_count(paragraph, regex):
    #
    #input: paragraph(string), regex(processed regex form of a single vocabulary)
    #output: count(int)
    #
    wordcount = Counter(re.findall(regex, paragraph))
    return sum(wordcount.values())

def liwc_words_count_dictionary(text, vocab_all, vocab_id):
    #
    #input: text@list of strings, vocab_all@list of regex, vocab_id@list of keys
    #output: list_of_dict@list of dictionaries, each dictionary corresponds to one paragraph
    #
    list_of_dict = []
    for i in range(len(text)):
        d = dict()
        for j in range(len(vocab_id)):
            key = vocab_id[j]
            value = liwc_words_count(text[i],vocab_all[j])
            if value != 0:
                d[key] = value
        list_of_dict.append(d)
    return list_of_dict

os.chdir('/Users/muriel820/Downloads/ml-project/cleaned')
zipfiles = glob('*zip')

for zfname in zipfiles:        
    print(zfname)
    zfile = ZipFile(zfname)    
    year = zfname.split('/')[-1][:-4]
 
    members = zfile.namelist()        
    threshold = len(members) / 200    
    docfreqs = Counter()

    zip_name = '/Users/muriel820/Downloads/ml-project/cleaned/liwc_results/' + zfname[0:4]+'/'
    #maj ='maj'
    os.makedirs(zip_name)
    #os.chdir(zip_name)
    os.makedirs(zip_name+'maj/')
    os.makedirs(zip_name+'con/')
    os.makedirs(zip_name+'dis/')
    os.makedirs(zip_name+'condis/')
    
    for fname in members:
        # "maj" means this is the majority opinion
        #print(fname)
        if fname.endswith('-maj.p'):
            docid = fname.split('/')[-1][:-2]                     
            text = pickle.load(zfile.open(fname,'r'))
            d = liwc_words_count_dictionary(text, vocab_all, vocab_id)
            name = docid + '.p'        
            pickle.dump(d, open(zip_name+'maj/'+name, "wb"))
        
        elif fname.endswith('.p')==False:
            continue 
        elif fname.endswith('dis/.p')==True:
            continue
        else:
            optype = fname.split('-')[-1][:-2]
            docid = fname.split('/')[-1][:-2]


            if len(optype) ==7:        
                text = pickle.load(zfile.open(fname,'r'))
                d = liwc_words_count_dictionary(text, vocab_all, vocab_id)
                name = docid + '.p'        
                pickle.dump(d, open(zip_name+'condis/'+name, "wb"))


            elif len(optype) == 4 and optype[0] =='c':
                text = pickle.load(zfile.open(fname,'r'))
                d = liwc_words_count_dictionary(text, vocab_all, vocab_id)
                name = docid + '.p'        
                pickle.dump(d, open(zip_name+'con/'+name, "wb"))

            else:
                text = pickle.load(zfile.open(fname,'r'))
                d = liwc_words_count_dictionary(text, vocab_all, vocab_id)
                name = docid + '.p'        
                pickle.dump(d, open(zip_name+'dis/'+name, "wb"))
    
    shutil.make_archive(zip_name, 'zip', zip_name)
    shutil.rmtree(zip_name,ignore_errors=True, onerror=None)

1908.zip
1909.zip
1910.zip
1911.zip
1912.zip
1913.zip
1914.zip
1915.zip
1916.zip
1917.zip
1918.zip
1919.zip
1920.zip
1921.zip
1922.zip
1923.zip
1924.zip
1925.zip
1926.zip
1927.zip
1928.zip
1929.zip
1930.zip
1931.zip
1932.zip
1933.zip
1934.zip
1935.zip
1936.zip
1937.zip
1938.zip
1939.zip
1940.zip
1941.zip
1942.zip
1943.zip
1944.zip
1945.zip
1946.zip
1947.zip
1948.zip
1949.zip
1950.zip
1951.zip
1952.zip
1953.zip
1954.zip
1955.zip
1956.zip
1957.zip
1958.zip
1959.zip
1960.zip
1961.zip
1962.zip
1963.zip
1964.zip
1965.zip
1966.zip


KeyboardInterrupt: 

## An example of output (we threw away 0s entry)

In [72]:
pickle.load(open('/Users/muriel820/Downloads/ml-project/cleaned/liwc_results/1880/maj/XOKFPRQNB5G0-maj.p','rb'))

[{'001-functi': 0,
  '002-prono': 9,
  '003-ppr': 2,
  '004': 1,
  '005-': 0,
  '006-y': 0,
  '007-she': 0,
  '008-th': 1,
  '009-ipr': 7,
  '010-artic': 28,
  '011-pr': 39,
  '012-auxve': 13,
  '013-adve': 4,
  '014-co': 16,
  '015-nega': 3,
  '020-ve': 0,
  '021-a': 0,
  '022-compa': 0,
  '023-interr': 2,
  '024-numb': 0,
  '025-qua': 1,
  '030-affe': 0,
  '031-pose': 0,
  '032-nege': 0,
  '033-a': 1,
  '034-ang': 1,
  '035-s': 0,
  '040-soci': 0,
  '041-fami': 0,
  '042-frie': 0,
  '043-fema': 0,
  '044-ma': 0,
  '050-cogpr': 0,
  '051-insig': 2,
  '052-cau': 3,
  '053-discr': 4,
  '054-tent': 6,
  '055-certa': 6,
  '056-diff': 7,
  '060-perce': 0,
  '061-s': 1,
  '062-he': 0,
  '063-fe': 1,
  '070-b': 0,
  '071-bo': 0,
  '072-heal': 0,
  '073-sexu': 0,
  '074-inge': 1,
  '080-driv': 0,
  '081-affiliati': 0,
  '082-achie': 0,
  '083-pow': 0,
  '084-rewa': 1,
  '085-ri': 1,
  '090-focuspa': 0,
  '091-focusprese': 0,
  '092-focusfutu': 1,
  '100-relat': 0,
  '101-moti': 0,
  '102-spa'